In [1]:
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from scipy.sparse.linalg import svds
from surprise import Reader
from surprise import Dataset
import pandas as pd
import numpy as np

rs = 0
np.random.seed(rs)

# Collaborative Filtering Recommender System
In this lab session, we will work with the training set created last week.

In [2]:
# load data
df = pd.read_pickle("train.pkl")
df = df[["overall", "reviewerID", "asin"]]
# test = pd.read_pickle("test.pkl")

## Exercise 1
In this exercise, we are going to predict the rating of a single user-item pair using a neighborhood-based method.
### 1.1
- Represent the ratings from the training set in a user-item matrix where the rows represent users and the columns represent items.
- Fill unobserved ratings with $0$.

Compute the cosine similarities between the user with 'reviewerID'='A25C2M3QF9G7OQ' and all users that have rated the item with 'asin'='B00EYZY6LQ'.<br>
What are the similarities and what are the ratings given by these users on item 'B00EYZY6LQ'?

In [3]:
# cc = users.groupby(['reviewerID']).cumcount() + 1
# df1 = users.set_index(['reviewerID', cc]).unstack().sort_index(1, level=1)
# df1.columns = ['_'.join(map(str,i)) for i in df1.columns]
# df1.reset_index()
# without_asin = users.drop(["asin"],axis=1)
# cc = without_asin.groupby(['reviewerID']).cumcount() + 1
# df1 = without_asin.set_index(['reviewerID', cc]).unstack().sort_index(1, level=1)
# df1.columns = ['_'.join(map(str,i)) for i in df1.columns]
# df1 = df1.reset_index().drop("reviewerID",axis=1).fillna(0)
# df1,df1.shape

In [4]:
user1 = df.loc[df['reviewerID'] == "A25C2M3QF9G7OQ"]
ids = df.loc[df['asin'] == "B00EYZY6LQ"]["reviewerID"]
users = df.loc[df['reviewerID'].isin(ids)]

u1 = user1.pivot_table(values="overall", index="reviewerID", columns="asin")
df2 = users.pivot_table(values="overall", index="reviewerID", columns="asin")
df2 = df2.append(u1).fillna(0)
df2
# df2.loc[df2['reviewerID'].isin(ids)]
# df2.mask(df2.apply(lambda x: all(x.name == d for d in (ids))))


asin,B000FOI48G,B000GLRREU,B000NKJIXM,B0010ZBORW,B0013NB7DW,B001E96LUO,B001ET7FZE,B001F51RAG,B001LNODUS,B002GP80EU,...,B00EF1QRMU,B00EYZY6LQ,B00L1I1VMG,B00N2WQ2IW,B00W259T7G,B016V8YWBC,B019809F9Y,B019FWRG3C,B01BNEYGQU,B01E7UKR38
reviewerID,,,,,,,,,,,,,,,,,,,,,
A1F7YU6O5RU432,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
A1R1BFJCMWX0Y3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0
A1UQBFCERIP7VJ,0.0,0.0,4.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,5.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0
A22CW0ZHY3NJH8,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0,4.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
A2LW5AL0KQ9P1M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,2.0,4.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
A2PD27UKAD3Q00,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2WW57XX2UVLM6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ZY49IDE6TY5I,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0
A39WWMBA0299ZF,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [5]:
sims = cosine_similarity(df2)[-1][:-1]
sims

array([0.07924289, 0.24514517, 0.05863363, 0.20788269, 0.27580986,
       0.        , 0.        , 0.68283473, 0.        , 0.        ,
       0.        , 0.24514517, 0.10567049])

### 1.2
Predict the rating for user 'A25C2M3QF9G7OQ' on item 'B00EYZY6LQ' based on the ratings from the $3$ most similar users, using a weighted (by similarity) average. What is the prediction?

In [6]:
ids = sims.argsort()[::-1][:3]
df2["B00EYZY6LQ"][ids] @ (sims[ids] / np.sum(sims[ids]))

4.203644504587891

## Exercise 2
In this exercise, we are going to predict the rating of the same user-item pair as in exercise 1, now using a latent factor method.
### 2.1
- Represent the ratings from the training set in a user-item matrix where the rows represent users and the columns represent items.
- Subtract the row mean (i.e. mean rating per user) from each non-missing element in the matrix.
- Replace missing values with $0$.

Factorize the user-item matrix by performing Singular Value Decomposition (SVD) of rank $5$ using eigendecomposition. What is ther user factors of user 'A25C2M3QF9G7OQ' and the item factors of item 'B00EYZY6LQ'?

In [7]:
df3 = df2.replace(0,np.nan)
means = df3.mean(1, skipna=True).values
df3 = df3.sub(means, axis=0).fillna(0)
df3
# df3.mean(1, skipna=True) # sanity check

asin,B000FOI48G,B000GLRREU,B000NKJIXM,B0010ZBORW,B0013NB7DW,B001E96LUO,B001ET7FZE,B001F51RAG,B001LNODUS,B002GP80EU,...,B00EF1QRMU,B00EYZY6LQ,B00L1I1VMG,B00N2WQ2IW,B00W259T7G,B016V8YWBC,B019809F9Y,B019FWRG3C,B01BNEYGQU,B01E7UKR38
reviewerID,,,,,,,,,,,,,,,,,,,,,
A1F7YU6O5RU432,0.0,0.0,-0.400000,0.000000,0.0,-0.400000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.600000,0.600000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.400000
A1R1BFJCMWX0Y3,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
A1UQBFCERIP7VJ,0.0,0.0,-0.333333,-2.333333,0.0,0.000000,0.666667,0.000000,0.000000,0.000000,...,0.000000,0.666667,0.000000,0.666667,0.0,0.666667,0.0,0.000000,0.666667,0.000000
A22CW0ZHY3NJH8,0.0,0.0,0.000000,0.000000,0.0,0.166667,0.166667,0.166667,0.000000,0.166667,...,0.000000,-0.833333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.166667,0.000000
A2LW5AL0KQ9P1M,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.500000,...,-1.500000,0.500000,-0.500000,0.000000,1.5,0.000000,0.0,0.000000,0.000000,0.000000
A2PD27UKAD3Q00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
A2WW57XX2UVLM6,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.333333,0.666667,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
A2ZY49IDE6TY5I,0.2,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.800000,0.000000,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.200000
A39WWMBA0299ZF,0.0,0.0,0.000000,-2.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.000000


In [8]:
# A25C2M3QF9G7OQ' and the item factors of item 'B00EYZY6LQ
u = df3.index.get_loc("A25C2M3QF9G7OQ")
v = df3.columns.get_loc("B00EYZY6LQ")
u,v


(13, 15)

In [9]:
Q, sigma, P = svds(df3, k=5)

S = np.zeros((5,5))
np.fill_diagonal(S, sigma)

Q.shape, P.shape
(Q@S)[u], P[:,v]

(array([ 0.1850614 , -0.45234499, -0.09669401, -2.59543051,  0.30967514]),
 array([ 0.15644228, -0.61357596,  0.45647152,  0.14384861,  0.41727102]))

### 2.2
Predict the rating for user 'A25C2M3QF9G7OQ' on item 'B00EYZY6LQ' by taking the dot product between the user factors and item factors and adding back the mean rating of this user. What is the prediction?

In [10]:
np.dot((Q@S)[u], P[:,v]) + means[-1]

4.418230759480559

## Exercise 3
### 3.1
Define a user-based neighborhood model that takes into account the mean rating of each user.<br>
Use cosine as similarity measure and try to vary the (maximum) number of neighbors to take into account when predicting ratings. Keep Scikit-Surprise's default setting for all other parameters. <br>
Is it better to use $1$ or $10$ neighbors? You should determine this based on the Root Mean Square Error (RMSE) over 3-fold cross-validation.

In [11]:
from surprise.model_selection.validation import cross_validate
from surprise import SVD
from surprise import KNNWithMeans

reader = Reader(rating_scale=(1, 5))
training = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

sim_options = {'name': 'cosine',
               'user_based': True 
               }

In [12]:
# Run 5-fold cross-validation and print results.
cross_validate(KNNWithMeans(k=1, sim_options=sim_options), training, measures=['RMSE'], cv=3, verbose=True)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.3693  0.4752  0.4006  0.4151  0.0444  
Fit time          0.28    0.32    0.35    0.32    0.03    
Test time         0.14    0.13    0.12    0.13    0.01    


{'test_rmse': array([0.36932648, 0.47522554, 0.40063765]),
 'fit_time': (0.2815549373626709, 0.3151881694793701, 0.35372400283813477),
 'test_time': (0.13802289962768555, 0.1331040859222412, 0.12088537216186523)}

In [13]:
cross_validate(KNNWithMeans(k=10, sim_options=sim_options), training, measures=['RMSE'], cv=3, verbose=True)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.4766  0.4377  0.3896  0.4346  0.0356  
Fit time          0.45    0.36    0.33    0.38    0.05    
Test time         0.24    0.15    0.12    0.17    0.05    


{'test_rmse': array([0.47662701, 0.4377063 , 0.38961484]),
 'fit_time': (0.45171236991882324, 0.3612394332885742, 0.3311796188354492),
 'test_time': (0.23576593399047852, 0.14520859718322754, 0.11888909339904785)}

In [14]:
# param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
# gs.fit(data)
# print(gs.best_score['rmse']) # best RMSE score
# print(gs.best_params['rmse'])# combination of parameters that gave the best RMSE score


### 3.2
Fit the neigborhood-based model defined in exercise 3.1 on the full training set with cosine as similarity measure and either $1$ or $10$ neighbors based on what you found to be better in exercise 3.1. Keep Scikit-Surprise's default setting for all other parameters, but set the random state to $0$ for comparable results. <br>
Use the model to predict the unobserved ratings for the users in the training set. How many predictions are there and what is the average of all the predictions?

In [15]:
trainset = training.build_full_trainset()
tmp = df.pivot_table(values="overall", index="reviewerID", columns="asin", aggfunc="first")
tmp.isna().sum().sum(), len(trainset.build_anti_testset())
# u, i = np.where(test.isna())
# u, i

(72404, 72404)

In [16]:
knn = KNNWithMeans(k=10, sim_options=sim_options, random_state=rs).fit(trainset)

test = trainset.build_anti_testset()

preds = pd.DataFrame(knn.test(test, verbose=False))
len(preds), preds["est"].mean()

Computing the cosine similarity matrix...
Done computing similarity matrix.


(72404, 4.6713656803782175)

## Exercise 4
### 4.1
Define an SVD model with user and item biases that uses Stochastic Gradient Descend (SGD) to estimate the low-rank matrix based on only observed ratings. <br>
Set the number of latent factors to $30$ and try to iterate the SGD procedure for different number of epochs. Keep Scikit-Surprise's default setting for all other parameters. <br>
Is it better to run for $100$ or $500$ epochs? You should determine this based on the RMSE over 3-fold cross-validation.

In [17]:
cross_validate(SVD(n_factors=30, n_epochs=100), training, measures=['RMSE'], cv=3, verbose=True)


Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.3048  0.5258  0.3796  0.4034  0.0918  
Fit time          0.32    0.28    0.29    0.29    0.02    
Test time         0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([0.30480044, 0.52583485, 0.3795723 ]),
 'fit_time': (0.3160884380340576, 0.279461145401001, 0.2874610424041748),
 'test_time': (0.006354808807373047,
  0.005910634994506836,
  0.006090402603149414)}

In [18]:
cross_validate(SVD(n_factors=30, n_epochs=500), training, measures=['RMSE'], cv=3, verbose=True)


Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.3271  0.4037  0.3035  0.3448  0.0428  
Fit time          1.63    1.50    1.29    1.47    0.14    
Test time         0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([0.32713468, 0.40365112, 0.30347318]),
 'fit_time': (1.6343486309051514, 1.4952259063720703, 1.2871363162994385),
 'test_time': (0.009630203247070312,
  0.0059185028076171875,
  0.006161928176879883)}

### 4.2
Fit the latent factor model defined in exercise 4.1 on the full training set with $30$ latent factors and run for either $100$ or $500$ epochs based on what you found to be better in exercise 4.1. Keep Scikit-Surprise's default setting for all other parameters, but set the random state to $0$ for comparable results.<br>
Use the model to predict the unobserved ratings for the users in the training set. How many predictions are there and what is the average of all the predictions?

In [19]:
svd = SVD(n_factors=30, n_epochs=500, random_state=0).fit(trainset)
preds2 = pd.DataFrame(svd.test(test, verbose=False))
len(preds2), preds2["est"].mean()

(72404, 4.504324196945566)

# Write data

In [20]:
df_test = pd.read_pickle("test.pkl")
dft = Dataset.load_from_df(df_test[['reviewerID', 'asin', 'overall']], reader)
dft = dft.build_full_trainset()
dft = dft.build_testset()

In [21]:
# preds1 = pd.DataFrame(knn.test(dft))
# preds2 = pd.DataFrame(svd.test(dft))


In [22]:
testset = pd.DataFrame(df_test)
testset = testset.rename(columns={0: "reviewerID", 1: "asin", 2: "overall"})

testset.to_pickle("testset.pkl")
preds.to_pickle("preds1.pkl")
preds2.to_pickle("preds2.pkl")

In [24]:
(preds["est"] - preds2["est"]).sum()

12094.471566457702